In [ ]:
import numpy as np
import pandas as pd

import os
import pprint
import tempfile

from typing import Dict, Text

import numpy as np
import tensorflow as tf

!pip install -q implicit

from itertools import islice, cycle
from more_itertools import pairwise
from implicit.nearest_neighbours import TFIDFRecommender

!pip install -q tensorflow-recommenders

import tensorflow_recommenders as tfrs

In [ ]:
import ast

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Load Data

In [ ]:
users_df = pd.read_csv(r"/content/drive/My Drive/DDH_2024/1_Task/data/preprocessed/users.csv")
inter_df = pd.read_csv(r"/content/drive/My Drive/DDH_2024/1_Task/data/preprocessed/interactions.csv")
items_df = pd.read_csv(r"/content/drive/My Drive/DDH_2024/1_Task/data/preprocessed/items.csv")

In [ ]:
inter_df=inter_df.astype('str')
items_df=items_df.astype('str')

In [ ]:
inter_df = pd.merge(inter_df, items_df, on="item_id")
inter_df=inter_df.astype('str')

In [ ]:
inter_df

,user_id,item_id,last_watch_dt,total_dur,watched_pct,content_type,title_orig,release_year,genres,countries,age_rating,description,keywords,actors_translated,actors_transliterated,directors_translated,transliterated
0,176549,9506,2021-05-11,4250,72.0,film,Frozen,2013.0,"fantasy, cartoon, musical",USA,0.0,When an ancient prophecy comes true and the ki...,"Queen, musical, princess, betrayal, snowman, d...","Kristen Bell, Idina Menzel, Jonathan Groff, Jo...","Kristen Bell, Idina Menzel, Dzhonatan Groff, D...","Chris Buck, Jennifer Lee","Kris Bak, Dzhennifer Li"
1,964868,9506,2021-04-30,6725,100.0,film,Frozen,2013.0,"fantasy, cartoon, musical",USA,0.0,When an ancient prophecy comes true and the ki...,"Queen, musical, princess, betrayal, snowman, d...","Kristen Bell, Idina Menzel, Jonathan Groff, Jo...","Kristen Bell, Idina Menzel, Dzhonatan Groff, D...","Chris Buck, Jennifer Lee","Kris Bak, Dzhennifer Li"
2,450170,9506,2021-07-18,848,14.0,film,Frozen,2013.0,"fantasy, cartoon, musical",USA,0.0,When an ancient prophecy comes true and the ki...,"Queen, musical, princess, betrayal, snowman, d...","Kristen Bell, Idina Menzel, Jonathan Groff, Jo...","Kristen Bell, Idina Menzel, Dzhonatan Groff, D...","Chris Buck, Jennifer Lee","Kris Bak, Dzhennifer Li"
3,683820,9506,2021-06-28,2617,44.0,film,Frozen,2013.0,"fantasy, cartoon, musical",USA,0.0,When an ancient prophecy comes true and the ki...,"Queen, musical, princess, betrayal, snowman, d...","Kristen Bell, Idina Menzel, Jonathan Groff, Jo...","Kristen Bell, Idina Menzel, Dzhonatan Groff, D...","Chris Buck, Jennifer Lee","Kris Bak, Dzhennifer Li"
4,657320,9506,2021-06-18,1070,18.0,film,Frozen,2013.0,"fantasy, cartoon, musical",USA,0.0,When an ancient prophecy comes true and the ki...,"Queen, musical, princess, betrayal, snowman, d...","Kristen Bell, Idina Menzel, Jonathan Groff, Jo...","Kristen Bell, Idina Menzel, Dzhonatan Groff, D...","Chris Buck, Jennifer Lee","Kris Bak, Dzhennifer Li"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4027317,352033,9125,2021-05-12,4431,19.0,series,The Adventures of Rasmus the Bear,2018.0,"children, tv series, adventure, foreign, weste...",Germany,0.0,Bear Rasmus is a very active and kind characte...,"Adventures, Bear, Rasmus, 2018, Germany, frien...","Werner Bönke, Gerald Schaale, Christian Zeiger...","Verner Benke, Geral'd Shaale, Kristian Tsajger...","Johannes Weiland, Carsten Bunte, Michael Bonen...","Johannes Vejland, Karsten Bunte, Mihael' Bonen..."
4027318,619743,13516,2021-07-11,14985,27.0,series,Animals.,2016.0,"animation, comedy",USA,18.0,"Comedy animated fantasy from the creators of ""...","Beasts, 2016, USA","Catherine Aselton, Mike Luciano, Megan O'Neill...","Ketrin Aselton, Majk Luchiano, Megan O’Nill, M...","Mike Luciano, Wesley Archer","Majk Luchiano, Uesli Archer"
4027319,797948,8030,2021-04-20,792,14.0,film,Madame Hyde,2017.0,"science fiction, foreign, comedy, screenplays",France,16.0,The actress Isabelle Huppert as Madame Jekyll'...,"Mrs, Hyde, 2017, France, revenge, science, tec...","Adda Senani, Angela Metzger, Guillaume Verdier...","Adda Senani, Anzhela Metsger, Gijom Verd'e, Iz...",Serge Bozon,Serzh Bozon
4027320,846909,13019,2021-03-13,10,0.0,film,Joséphine (Sign Language),2013.0,"foreign, comedy",France,12.0,"When even her sister marries, Josephine breaks...","Want Like, Bridget, Sign Language, 2013, France","Marilou Berry, Mehdi Nebbu, Alice Paul, Berang...","Marilu Berri, Mehdi Nebbu, Alis Pol', Beranzhe...",Añez Obadia,An'es Obad'ja


In [ ]:
#inter_df=inter_df[:100000]

In [ ]:
def df_to_map_dataset(dataframe, batch_size=1):
  """
  Converts a Pandas DataFrame to a MapDataset in TensorFlow.

  Args:
    dataframe: A Pandas DataFrame.
    batch_size: The batch size for the MapDataset.

  Returns:
    A MapDataset containing the data from the DataFrame.
  """

  # Create a dictionary of column names and values.
  #data = {name: values for name, values in dataframe.items()}

  # Create a MapDataset from the dictionary.
  #dataset = tf.data.Dataset.from_tensor_slices(data)
  dataset = tf.data.Dataset.from_tensor_slices(dict(dataframe))

  # Batch the dataset.
  #dataset = dataset.batch(batch_size)

  return dataset

In [ ]:
inter_map=df_to_map_dataset(inter_df, batch_size=1)
items_map=df_to_map_dataset(items_df, batch_size=1)

In [ ]:
#for x in inter_map.take(1).as_numpy_iterator():
#  pprint.pprint(x)

In [ ]:
inters = inter_map.map(lambda x: {
    "title_orig": x["title_orig"],
    "user_id": x["user_id"],
    'actors_translated': x['actors_translated'],
    'age_rating': x['age_rating'],
    'description': x['description']
})
items = items_map.map(lambda x: x["title_orig"])

In [ ]:
for x in inters.take(1).as_numpy_iterator():
  pprint.pprint(x)

{'actors_translated': b'Kristen Bell, Idina Menzel, Jonathan Groff, Josh Gad'
                      b', Santino Fontana, Alan Tewdik, Kieran Hinds, Chris '
                      b'Williams, Stephen J. Anderson, Maya Wilson, Ciaran H'
                      b'inds, Maurice LaMarsh, Jennifer Lee, Dara McGarry, F'
                      b'red Tatasquior',
 'age_rating': b'0.0',
 'description': b'When an ancient prophecy comes true and the kingdom is plung'
                b'ed into the embrace of eternal winter, three fearless heroes'
                b' - Princess Anne, the brave Kristoff, and his faithful reind'
                b"eer Sven - set out to find Anna's sister, Elsa, who can lift"
                b" the country's icy spell. Along the way, they will face a ho"
                b'st of fascinating surprises and exciting adventures: meeting'
                b' mystical trolls, meeting a charming snowman named Olaf, cli'
                b'mbing mountains steeper than Mount Everest, an

In [ ]:
for x in items.take(2).as_numpy_iterator():
  pprint.pprint(x)

b'Hable con ella'
b'Search Party'


In [ ]:
user_ids_vocabulary = tf.keras.layers.StringLookup(mask_token=None)
user_ids_vocabulary.adapt(inters.map(lambda x: x["user_id"]))

items_vocabulary = tf.keras.layers.StringLookup(mask_token=None)
items_vocabulary.adapt(items)

In [ ]:
class MovieLensModel(tfrs.Model):
  # We derive from a custom base class to help reduce boilerplate. Under the hood,
  # these are still plain Keras Models.

  def __init__(
      self,
      user_model: tf.keras.Model,
      movie_model: tf.keras.Model,
      task: tfrs.tasks.Retrieval):
    super().__init__()

    # Set up user and movie representations.
    self.user_model = user_model
    self.movie_model = movie_model

    # Set up a retrieval task.
    self.task = task

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # Define how the loss is computed.

    user_embeddings = self.user_model(features["user_id"])
    movie_embeddings = self.movie_model(features["title_orig"])

    return self.task(user_embeddings, movie_embeddings)

In [ ]:
# Define user and movie models.
user_model = tf.keras.Sequential([user_ids_vocabulary, tf.keras.layers.Embedding(user_ids_vocabulary.vocabulary_size(), 64)])
movie_model = tf.keras.Sequential([items_vocabulary, tf.keras.layers.Embedding(items_vocabulary.vocabulary_size(), 64)])

# Define your objectives.
task = tfrs.tasks.Retrieval(metrics=tfrs.metrics.FactorizedTopK(items.batch(128).map(movie_model)))

In [ ]:
user_model.save(r"/content/drive/My Drive/DDH_2024/1_Task/models/user_model_3_exp.keras")
movie_model.save(r"/content/drive/My Drive/DDH_2024/1_Task/models/movie_model_3_exp.keras")

/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_api.py:164: UserWarning: You are saving a model that has not yet been built. It might not contain any weights yet. Consider building the model first by calling it on some data.
  saving_lib.save_model(model, filepath)


In [ ]:
user_model_loaded = tf.keras.models.load_model(r"/content/drive/My Drive/DDH_2024/1_Task/models/user_model_1_exp.keras")
movie_model_loaded = tf.keras.models.load_model(r"/content/drive/My Drive/DDH_2024/1_Task/models/movie_model_1_exp.keras")

In [ ]:
# Create a retrieval model.
model = MovieLensModel(user_model, movie_model, task)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.5))

# Train for 3 epochs.
model.fit(inters.batch(4096), epochs=1)

# Use brute-force search to set up retrieval using the trained representations.
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
index.index_from_dataset(items.batch(100).map(lambda title: (title, model.movie_model(title))))

972/972 [==============================] - 292s 299ms/step - factorized_top_k/top_1_categorical_accuracy: 2.5386e-05 - factorized_top_k/top_5_categorical_accuracy: 0.0011 - factorized_top_k/top_10_categorical_accuracy: 0.0023 - factorized_top_k/top_50_categorical_accuracy: 0.0117 - factorized_top_k/top_100_categorical_accuracy: 0.0229 - loss: 34052.5367 - regularization_loss: 0.0000e+00 - total_loss: 34052.5367


In [ ]:
model.save_weights(r"/content/drive/My Drive/DDH_2024/1_Task/models/weights/MLM_1_exp", save_format='tf')

In [ ]:
task = tfrs.tasks.Retrieval(metrics=tfrs.metrics.FactorizedTopK(items.batch(128).map(movie_model_loaded)))
loaded_model = MovieLensModel(user_model_loaded, movie_model_loaded, task)
loaded_model.load_weights(r"/content/drive/My Drive/DDH_2024/1_Task/models/weights/MLM_1_exp")

# Use brute-force search to set up retrieval using the trained representations.
index = tfrs.layers.factorized_top_k.BruteForce(loaded_model.user_model)
index.index_from_dataset(items.batch(100).map(lambda title: (title, loaded_model.movie_model(title))))

In [ ]:

# Get some recommendations.
_, titles = index(np.array(["10"]))
print(f"Top 10 recommendations for user 1097558: {titles[0, :11]}")

Top 10 recommendations for user 1097558: [b'Declared Dead' b'A Dark, Dark Man' b'\xc3\x89pid\xc3\xa9mie'
 b'Awareness' b'Sacrifice' b'Farewell' b'Occupation' b'Lapsi' b'The Plant'
 b'Anatomy of a Murder']


# Load test data



In [ ]:
sample_submission = pd.read_csv(r"/content/drive/My Drive/DDH_2024/1_Task/data/raw/sample_submission.csv")

In [ ]:
predicted_movies=dict()
for user_id in sample_submission.user_id.values:
  _, predicted_movies[user_id] = index(np.array([str(user_id)]))

In [ ]:
predicted_ids=list()
predicted_dict=dict()
for user_id in predicted_movies.keys():
  for movie in predicted_movies[user_id].numpy()[0]:
    predicted_ids.append(items_df[items_df['title_orig']==movie.decode("utf-8")]['item_id'].values[0])
  predicted_dict[user_id]=predicted_ids
  predicted_ids=list()

In [ ]:
predicted_dict_fordf=dict()
for key in predicted_dict.keys():
  predicted_dict_fordf['user_id']=key
  predicted_dict_fordf['item_id']=predicted_dict[key]

In [ ]:
predicted_df=pd.DataFrame(predicted_dict)

In [ ]:
df_melted = predicted_df.melt(var_name='user_id', value_name='item_id')

In [ ]:
df_grouped = df_melted.groupby('user_id')['item_id'].apply(list).reset_index()

# Метрики

In [ ]:
def calculate_novelty(train_interactions, recommendations, top_n):
    users = recommendations['user_id'].unique()
    n_users = train_interactions['user_id'].nunique()
    n_users_per_item = train_interactions.groupby('item_id')['user_id'].nunique()

    recommendations = recommendations.loc[recommendations['rank'] <= top_n].copy()
    recommendations['n_users_per_item'] = recommendations['item_id'].map(n_users_per_item)
    recommendations['n_users_per_item'] = recommendations['n_users_per_item'].fillna(1)
    recommendations['item_novelty'] = -np.log2(recommendations['n_users_per_item'] / n_users)

    item_novelties = recommendations[['user_id', 'rank', 'item_novelty']]

    miuf_at_k = item_novelties.loc[item_novelties['rank'] <= top_n, ['user_id', 'item_novelty']]
    miuf_at_k = miuf_at_k.groupby('user_id').agg('mean').squeeze()

    return miuf_at_k.reindex(users).mean()

In [ ]:
def compute_metrics(train, test, recs, top_N):
    result = {}
    test_recs = test.set_index(['user_id', 'item_id']).join(recs.set_index(['user_id', 'item_id']))
    test_recs = test_recs.sort_values(by=['user_id', 'rank'])

    test_recs['users_item_count'] = test_recs.groupby(level='user_id')['rank'].transform(np.size)
    test_recs['reciprocal_rank'] = (1 / test_recs['rank']).fillna(0)
    test_recs['cumulative_rank'] = test_recs.groupby(level='user_id').cumcount() + 1
    test_recs['cumulative_rank'] = test_recs['cumulative_rank'] / test_recs['rank']

    users_count = test_recs.index.get_level_values('user_id').nunique()

    # Uncomment for Precision/Recall at k results

#    for k in range(1, top_N + 1):
#         hit_k = f'hit@{k}'
#         test_recs[hit_k] = test_recs['rank'] <= k
#         result[f'Precision@{k}'] = (test_recs[hit_k] / k).sum() / users_count
#         result[f'Recall@{k}'] = (test_recs[hit_k] / test_recs['users_item_count']).sum() / users_count

    result[f'MAP@{top_N}'] = (test_recs['cumulative_rank'] / test_recs['users_item_count']).sum() / users_count
    result[f'Novelty@{top_N}'] = calculate_novelty(train, recs, top_N)

    return pd.Series(result)

In [ ]:
class TimeRangeSplit():
    """
        https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.date_range.html
    """
    def __init__(self,
                 start_date,
                 end_date=None,
                 freq='D',
                 periods=None,
                 tz=None,
                 normalize=False,
                 closed=None,
                 train_min_date=None,
                 filter_cold_users=True,
                 filter_cold_items=True,
                 filter_already_seen=True):

        self.start_date = start_date
        if end_date is None and periods is None:
            raise ValueError('Either "end_date" or "periods" must be non-zero, not both at the same time.')

        self.end_date = end_date
        self.freq = freq
        self.periods = periods
        self.tz = tz
        self.normalize = normalize
        self.closed = closed
        self.train_min_date = pd.to_datetime(train_min_date, errors='raise')
        self.filter_cold_users = filter_cold_users
        self.filter_cold_items = filter_cold_items
        self.filter_already_seen = filter_already_seen

        self.date_range = pd.date_range(
            start=start_date,
            end=end_date,
            freq=freq,
            periods=periods,
            tz=tz,
            normalize=normalize,
            closed=closed)

        self.max_n_splits = max(0, len(self.date_range) - 1)
        if self.max_n_splits == 0:
            raise ValueError('Provided parametrs set an empty date range.')

    def split(self,
              df,
              user_column='user_id',
              item_column='item_id',
              datetime_column='date',
              fold_stats=False):
        df_datetime = df[datetime_column]
        if self.train_min_date is not None:
            train_min_mask = df_datetime >= self.train_min_date
        else:
            train_min_mask = df_datetime.notnull()

        date_range = self.date_range[(self.date_range >= df_datetime.min()) &
                                     (self.date_range <= df_datetime.max())]

        for start, end in pairwise(date_range):
            fold_info = {
                'Start date': start,
                'End date': end
            }
            train_mask = train_min_mask & (df_datetime < start)
            train_idx = df.index[train_mask]
            if fold_stats:
                fold_info['Train'] = len(train_idx)

            test_mask = (df_datetime >= start) & (df_datetime < end)
            test_idx = df.index[test_mask]

            if self.filter_cold_users:
                new = np.setdiff1d(
                    df.loc[test_idx, user_column].unique(),
                    df.loc[train_idx, user_column].unique())
                new_idx = df.index[test_mask & df[user_column].isin(new)]
                test_idx = np.setdiff1d(test_idx, new_idx)
                test_mask = df.index.isin(test_idx)
                if fold_stats:
                    fold_info['New users'] = len(new)
                    fold_info['New users interactions'] = len(new_idx)

            if self.filter_cold_items:
                new = np.setdiff1d(
                    df.loc[test_idx, item_column].unique(),
                    df.loc[train_idx, item_column].unique())
                new_idx = df.index[test_mask & df[item_column].isin(new)]
                test_idx = np.setdiff1d(test_idx, new_idx)
                test_mask = df.index.isin(test_idx)
                if fold_stats:
                    fold_info['New items'] = len(new)
                    fold_info['New items interactions'] = len(new_idx)

            if self.filter_already_seen:
                user_item = [user_column, item_column]
                train_pairs = df.loc[train_idx, user_item].set_index(user_item).index
                test_pairs = df.loc[test_idx, user_item].set_index(user_item).index
                intersection = train_pairs.intersection(test_pairs)
                print(f'Already seen number: {len(intersection)}')
                test_idx = test_idx[~test_pairs.isin(intersection)]
                # test_mask = rd.df.index.isin(test_idx)
                if fold_stats:
                    fold_info['Known interactions'] = len(intersection)

            if fold_stats:
                fold_info['Test'] = len(test_idx)

            yield (train_idx, test_idx, fold_info)

    def get_n_splits(self, df, datetime_column='date'):
        df_datetime = df[datetime_column]
        if self.train_min_date is not None:
            df_datetime = df_datetime[df_datetime >= self.train_min_date]

        date_range = self.date_range[(self.date_range >= df_datetime.min()) &
                                     (self.date_range <= df_datetime.max())]

        return max(0, len(date_range) - 1)

In [ ]:
def get_coo_matrix(df,
                   user_col='user_id',
                   item_col='item_id',
                   weight_col=None,
                   users_mapping={},
                   items_mapping={}):

    if weight_col is None:
        weights = np.ones(len(df), dtype=np.float32)
    else:
        weights = df[weight_col].astype(np.float32)

    interaction_matrix = sp.coo_matrix((
        weights,
        (
            df[user_col].map(users_mapping.get),
            df[item_col].map(items_mapping.get)
        )
    ))
    return interaction_matrix

In [ ]:
def generate_implicit_recs_mapper(
    model,
    train_matrix,
    top_N,
    user_mapping,
    item_inv_mapping,
    filter_already_liked_items
):
    def _recs_mapper(user):
        user_id = user_mapping[user]
        recs = model.recommend(user_id,
                               train_matrix,
                               N=top_N,
                               filter_already_liked_items=filter_already_liked_items)
        return [item_inv_mapping[item] for item, _ in recs]
    return _recs_mapper

In [ ]:
class PopularRecommender():
    def __init__(self, max_K=10, days=30, item_column='item_id', dt_column='date'):
        self.max_K = max_K
        self.days = days
        self.item_column = item_column
        self.dt_column = dt_column
        self.recommendations = []

    def fit(self, df, ):
        min_date = df[self.dt_column].max().normalize() - pd.DateOffset(days=self.days)
        self.recommendations = df.loc[df[self.dt_column] > min_date, self.item_column].value_counts().head(self.max_K).index.values

    def recommend(self, users=None, N=10):
        recs = self.recommendations[:N]
        if users is None:
            return recs
        else:
            return list(islice(cycle([recs]), len(users)))

In [ ]:
interactions_df = pd.read_csv(r"/content/drive/My Drive/DDH_2024/1_Task/data/preprocessed/interactions.csv", parse_dates=['last_watch_dt'])

In [ ]:
test = interactions_df[interactions_df['last_watch_dt'] == interactions_df['last_watch_dt'].max()]
train = interactions_df[interactions_df['last_watch_dt'] < interactions_df['last_watch_dt'].max()]

In [ ]:
pop_model = PopularRecommender(days=7, dt_column='last_watch_dt')
pop_model.fit(train)

In [ ]:
recs = pd.DataFrame({'user_id': test['user_id'].unique()})
top_N = 10
recs['item_id'] = pop_model.recommend(recs['user_id'], N=top_N)
recs.head()

In [ ]:
recs.shape, test.shape

((32352, 2), (53956, 5))

In [ ]:
def convert_to_int(lst):
    return [int(x) for x in lst]

# Преобразование всех элементов в столбце 'item_id' из строк в числа
df_grouped['item_id'] = df_grouped['item_id'].apply(convert_to_int)

In [ ]:
recs_df = df_grouped[df_grouped['user_id'].isin(recs['user_id'])]

In [ ]:
recs_df.shape

(18489, 2)

In [ ]:
recs.shape

(32352, 2)

In [ ]:
recs.user_id[0]

626036

In [ ]:
recs.item_id[0]

array([ 9728, 15297, 10440, 14488, 13865, 12192, 12360,   512,   341,
        3734])

In [ ]:
recs = df_grouped.explode('item_id')

In [ ]:
recs['rank'] = recs.groupby('user_id').cumcount() + 1
recs.head(top_N + 2)

,user_id,item_id,rank
0,3,10289,1
0,3,15592,2
0,3,2410,3
0,3,284,4
0,3,164,5
0,3,5990,6
0,3,9947,7
0,3,14109,8
0,3,8162,9
0,3,7185,10


In [ ]:
compute_metrics(interactions_df, test, recs, 10)

MAP@10         0.022929
Novelty@10    15.320492
dtype: float64